In [43]:
import numpy as np
from mtdlearn.mtd import MTD
from mtdlearn.preprocessing import PathEncoder
from mtdlearn.datasets import data_values3_order2_full
from sklearn.metrics import accuracy_score

In [25]:
x = data_values3_order2_full['x']
y = data_values3_order2_full['y']
sample_weight = data_values3_order2_full['sample_weight']

In [26]:
x[:5]

array([['A>A>A'],
       ['A>A>B'],
       ['A>A>C'],
       ['A>B>A'],
       ['A>B>A']], dtype='<U5')

In [27]:
y[:5]

array(['A', 'A', 'A', 'A', 'B'], dtype='<U1')

In [28]:
sample_weight[:5]

array([1000, 1000, 1000,  100,  100])

In [29]:
pe = PathEncoder(3)
pe.fit(x, y)

PathEncoder(order=3, r_just_string='null', sep='>')

In [30]:
pe.label_dict

{'A': 0, 'B': 1, 'C': 2, 'null': 3}

In [8]:
x_tr3, y_tr3 = pe.transform(x, y)

In [9]:
x_tr3[:5]

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 2],
       [0, 1, 0],
       [0, 1, 0]])

In [10]:
y_tr3[:5]

array([0, 0, 0, 0, 1])

In [31]:
model = MTD(n_dimensions=3, order=3)

In [32]:
model.fit(x_tr3, y_tr3)

best value: -81.85092905482018


In [33]:
model.bic

224.1466916991487

In [45]:
model.predict(pe.transform(np.array([['A>B>C'], ['B>B>A'], ['C>C>C']])))

array([0, 2, 0])

In [46]:
model.predict_proba(pe.transform(np.array([['A>B>C'], ['B>B>A'], ['C>C>C']])))

array([[0.39856197, 0.32330497, 0.27813305],
       [0.31221481, 0.32581974, 0.36196545],
       [0.33610053, 0.33222156, 0.33167791]])